In [21]:
import pandas as pd
import warnings

# Oculta todos os warnings
warnings.filterwarnings("ignore")

#### Carregando os dados originais

In [12]:
# Definir diretórios
dados_originais = r"dados/"

# Carregar os datasets
df_saude_mental = pd.read_csv(f"{dados_originais}Impacto_do_Trabalho_Remoto_na_Saude_Mental.csv")
df_bem_estar_feedback = pd.read_csv(f"{dados_originais}Programas_de BemEstar_e_Feedback_dos_Funcionários.csv")
df_prog_carreira = pd.read_csv(f"{dados_originais}Progresssao_de_Carreira_do_Funcionário.csv")

In [13]:
# Renomear colunas
df_saude_mental = df_saude_mental.rename(columns={
    "Employee_ID": "ID_Empregado",
    "Age": "Idade",
    "Gender": "Genero",
    "Job_Role": "Cargo",
    "Industry": "Industria",
    "Years_of_Experience": "Anos_Experiencia",
    "Work_Location": "Local_Trabalho",
    "Hours_Worked_Per_Week": "Horas_Trabalhadas_Semana",
    "Number_of_Virtual_Meetings": "Numero_Reunioes_Virtuais",
    "Work_Life_Balance_Rating": "Avaliacao_Equilibrio_Vida",
    "Stress_Level": "Nivel_Estresse",
    "Mental_Health_Condition": "Condicao_Saude_Mental",
    "Access_to_Mental_Health_Resources": "Acesso_Recursos_Saude_Mental",
    "Productivity_Change": "Mudanca_Produtividade",
    "Social_Isolation_Rating": "Avaliacao_Isolamento_Social",
    "Satisfaction_with_Remote_Work": "Satisfacao_Trabalho_Remoto",
    "Company_Support_for_Remote_Work": "Suporte_Empresa_Trabalho_Remoto",
    "Physical_Activity": "Atividade_Fisica",
    "Sleep_Quality": "Qualidade_Sono",
    "Region": "Regiao"
})

df_bem_estar_feedback = df_bem_estar_feedback.rename(columns={
    "Employee_ID": "ID_Empregado",
    "Program_ID": "ID_Programa",
    "Program_Name": "Nome_Programa",
    "Participation_Status": "Status_Participacao",
    "Feedback_Score": "Pontuacao_Feedback",
    "Improvement_Noticed": "Melhoria_Observada"
})

df_prog_carreira = df_prog_carreira.rename(columns={
    "Employee_ID": "ID_Empregado",
    "Promotion_Date": "Data_Promocao",
    "Old_Role": "Cargo_Anterior",
    "New_Role": "Novo_Cargo",
    "Salary_Change": "Mudanca_Salarial",
    "Role_Satisfaction": "Satisfacao_Cargo"
})

#### Exibindo informações básicas

In [18]:
# Exibir informações básicas
def exibir_info(df, nome):
    print(f"\n{nome} Info:")
    display(df.isnull().sum())
    display(df.head(5))

dfs = {"Saude Mental": df_saude_mental, "Bem-estar": df_bem_estar_feedback, "Progresso Carreira": df_prog_carreira}
for nome, df in dfs.items():
    exibir_info(df, nome)


Saude Mental Info:


ID_Empregado                          0
Idade                                 0
Genero                                0
Cargo                                 0
Industria                             0
Anos_Experiencia                      0
Local_Trabalho                        0
Horas_Trabalhadas_Semana              0
Numero_Reunioes_Virtuais              0
Avaliacao_Equilibrio_Vida             0
Nivel_Estresse                        0
Condicao_Saude_Mental              1196
Acesso_Recursos_Saude_Mental          0
Mudanca_Produtividade                 0
Avaliacao_Isolamento_Social           0
Satisfacao_Trabalho_Remoto            0
Suporte_Empresa_Trabalho_Remoto       0
Atividade_Fisica                   1629
Qualidade_Sono                        0
Regiao                                0
dtype: int64

,ID_Empregado,Idade,Genero,Cargo,Industria,Anos_Experiencia,Local_Trabalho,Horas_Trabalhadas_Semana,Numero_Reunioes_Virtuais,Avaliacao_Equilibrio_Vida,Nivel_Estresse,Condicao_Saude_Mental,Acesso_Recursos_Saude_Mental,Mudanca_Produtividade,Avaliacao_Isolamento_Social,Satisfacao_Trabalho_Remoto,Suporte_Empresa_Trabalho_Remoto,Atividade_Fisica,Qualidade_Sono,Regiao
0,EMP0001,32,Non-binary,HR,Healthcare,13,Hybrid,47,7,2,Medium,Depression,No,Decrease,1,Unsatisfied,1,Weekly,Good,Europe
1,EMP0002,40,Female,Data Scientist,IT,3,Remote,52,4,1,Medium,Anxiety,No,Increase,3,Satisfied,2,Weekly,Good,Asia
2,EMP0003,59,Non-binary,Software Engineer,Education,22,Hybrid,46,11,5,Medium,Anxiety,No,No Change,4,Unsatisfied,5,NaN,Poor,North America
3,EMP0004,27,Male,Software Engineer,Finance,20,Onsite,32,8,4,High,Depression,Yes,Increase,3,Unsatisfied,3,NaN,Poor,Europe
4,EMP0005,49,Male,Sales,Consulting,32,Onsite,35,12,2,High,NaN,Yes,Decrease,3,Unsatisfied,3,Weekly,Average,North America



Bem-estar Info:


ID_Empregado           0
ID_Programa            0
Nome_Programa          0
Status_Participacao    0
Pontuacao_Feedback     0
Melhoria_Observada     0
dtype: int64

,ID_Empregado,ID_Programa,Nome_Programa,Status_Participacao,Pontuacao_Feedback,Melhoria_Observada
0,EMP0861,4,Fitness Challenge,Participou,5,No
1,EMP3773,1,Mindfulness Meditation,Participou,1,No
2,EMP3093,2,Work-Life Balance Workshop,Participou,1,Yes
3,EMP0467,3,Healthy Eating Plan,Participou,5,Yes
4,EMP4427,2,Work-Life Balance Workshop,Participou,1,Yes



Progresso Carreira Info:


ID_Empregado        0
Data_Promocao       0
Cargo_Anterior      0
Novo_Cargo          0
Mudanca_Salarial    0
Satisfacao_Cargo    0
dtype: int64

,ID_Empregado,Data_Promocao,Cargo_Anterior,Novo_Cargo,Mudanca_Salarial,Satisfacao_Cargo
0,EMP1717,2018-02-28,Team Leader,Senior Developer,1157,5
1,EMP3203,2023-03-31,Senior Developer,Senior Developer,4856,1
2,EMP2858,2023-02-28,Senior Developer,Team Leader,1826,3
3,EMP0787,2026-04-30,Senior Developer,Team Leader,4465,5
4,EMP4369,2019-07-31,Senior Developer,Junior Developer,2890,3


In [15]:
# Exibir contagem de linhas e colunas (linhas, colunas)
print("Saúde Mental", df_saude_mental.shape)
print("Bem-estar", df_bem_estar_feedback.shape)
print("Progresso de Carreira", df_prog_carreira.shape)

Saúde Mental (5000, 20)
Bem-estar (1000, 6)
Progresso de Carreira (500, 6)


#### Tratando valores ausentes, removendo duplicatas e excluindo colunas irrelevantes.

In [22]:
# Tratamento de valores ausentes com pandas
def tratar_nulos_pandas(df):
    for col_name in df.columns:
        # Pular as colunas ID_Empregado e Data_Promocao
        if col_name in ["ID_Empregado", "Data_Promocao"]:
            continue
        
        # Verificar se a coluna não é do tipo string
        if df[col_name].dtype != 'object':
            # Calcular a média para colunas numéricas
            media_valor = df[col_name].mean()
            df[col_name].fillna(media_valor, inplace=True)
        else:
            # Preencher valores nulos com 'Desconhecido' para colunas de texto
            df[col_name].fillna('Desconhecido', inplace=True)
    
    return df

df_saude_mental = tratar_nulos_pandas(df_saude_mental)
df_bem_estar_feedback = tratar_nulos_pandas(df_bem_estar_feedback)
df_prog_carreira = tratar_nulos_pandas(df_prog_carreira)

In [23]:
dfs = {"Saude Mental": df_saude_mental, "Bem-estar": df_bem_estar_feedback, "Progresso Carreira": df_prog_carreira}
for nome, df in dfs.items():
    exibir_info(df, nome)


Saude Mental Info:


ID_Empregado                       0
Idade                              0
Genero                             0
Cargo                              0
Industria                          0
Anos_Experiencia                   0
Local_Trabalho                     0
Horas_Trabalhadas_Semana           0
Numero_Reunioes_Virtuais           0
Avaliacao_Equilibrio_Vida          0
Nivel_Estresse                     0
Condicao_Saude_Mental              0
Acesso_Recursos_Saude_Mental       0
Mudanca_Produtividade              0
Avaliacao_Isolamento_Social        0
Satisfacao_Trabalho_Remoto         0
Suporte_Empresa_Trabalho_Remoto    0
Atividade_Fisica                   0
Qualidade_Sono                     0
Regiao                             0
dtype: int64

,ID_Empregado,Idade,Genero,Cargo,Industria,Anos_Experiencia,Local_Trabalho,Horas_Trabalhadas_Semana,Numero_Reunioes_Virtuais,Avaliacao_Equilibrio_Vida,Nivel_Estresse,Condicao_Saude_Mental,Acesso_Recursos_Saude_Mental,Mudanca_Produtividade,Avaliacao_Isolamento_Social,Satisfacao_Trabalho_Remoto,Suporte_Empresa_Trabalho_Remoto,Atividade_Fisica,Qualidade_Sono,Regiao
0,EMP0001,32,Non-binary,HR,Healthcare,13,Hybrid,47,7,2,Medium,Depression,No,Decrease,1,Unsatisfied,1,Weekly,Good,Europe
1,EMP0002,40,Female,Data Scientist,IT,3,Remote,52,4,1,Medium,Anxiety,No,Increase,3,Satisfied,2,Weekly,Good,Asia
2,EMP0003,59,Non-binary,Software Engineer,Education,22,Hybrid,46,11,5,Medium,Anxiety,No,No Change,4,Unsatisfied,5,Desconhecido,Poor,North America
3,EMP0004,27,Male,Software Engineer,Finance,20,Onsite,32,8,4,High,Depression,Yes,Increase,3,Unsatisfied,3,Desconhecido,Poor,Europe
4,EMP0005,49,Male,Sales,Consulting,32,Onsite,35,12,2,High,Desconhecido,Yes,Decrease,3,Unsatisfied,3,Weekly,Average,North America



Bem-estar Info:


ID_Empregado           0
ID_Programa            0
Nome_Programa          0
Status_Participacao    0
Pontuacao_Feedback     0
Melhoria_Observada     0
dtype: int64

,ID_Empregado,ID_Programa,Nome_Programa,Status_Participacao,Pontuacao_Feedback,Melhoria_Observada
0,EMP0861,4,Fitness Challenge,Participou,5,No
1,EMP3773,1,Mindfulness Meditation,Participou,1,No
2,EMP3093,2,Work-Life Balance Workshop,Participou,1,Yes
3,EMP0467,3,Healthy Eating Plan,Participou,5,Yes
4,EMP4427,2,Work-Life Balance Workshop,Participou,1,Yes



Progresso Carreira Info:


ID_Empregado        0
Data_Promocao       0
Cargo_Anterior      0
Novo_Cargo          0
Mudanca_Salarial    0
Satisfacao_Cargo    0
dtype: int64

,ID_Empregado,Data_Promocao,Cargo_Anterior,Novo_Cargo,Mudanca_Salarial,Satisfacao_Cargo
0,EMP1717,2018-02-28,Team Leader,Senior Developer,1157,5
1,EMP3203,2023-03-31,Senior Developer,Senior Developer,4856,1
2,EMP2858,2023-02-28,Senior Developer,Team Leader,1826,3
3,EMP0787,2026-04-30,Senior Developer,Team Leader,4465,5
4,EMP4369,2019-07-31,Senior Developer,Junior Developer,2890,3


In [24]:
# Remover duplicatas
df_saude_mental = df_saude_mental.drop_duplicates()
df_bem_estar_feedback = df_bem_estar_feedback.drop_duplicates()
df_prog_carreira = df_prog_carreira.drop_duplicates()

#### Salvando os datasets depois da limpeza

In [26]:
# Salvar datasets limpos
import os

dados_processados = "dados_processados/"

if not os.path.exists(dados_processados):
    os.makedirs(dados_processados)

df_saude_mental.to_csv(f"{dados_processados}saude_mental_limpo.csv", index=False)
df_bem_estar_feedback.to_csv(f"{dados_processados}bem_estar_feedback_limpo.csv", index=False)
df_prog_carreira.to_csv(f"{dados_processados}prog_carreira_limpo.csv", index=False)